In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19

In [ ]:
!nvidia-smi

Fri May  5 07:25:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# LaMini-LM Neo 1.3B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import torch

# checkpoint = "MBZUAI/LaMini-Flan-T5-783M" 
checkpoint = "MBZUAI/LaMini-Neo-1.3B" 
# checkpoint = "MBZUAI/LaMini-GPT-1.5B" 



tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_8bit=True)

pipe = pipeline('text-generation', 
                 model = base_model,
                 tokenizer = tokenizer,
                 max_length=512, 
                 do_sample=True,
                 pad_token_id= 50256,
                 temperature=0.7,
                 top_p=0.95,
                 repetition_penalty=1.15
                 )


### The prompt & response

In [ ]:
import json
import textwrap

human_prompt = 'What is the meaning of life?'

def get_prompt(instruction):
    prompt_template = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:"
    return prompt_template

print(get_prompt('What is the meaning of life?'))

def parse_text(data):
    for item in data:
        text = item['generated_text']
        assistant_text_index = text.find('### Response:')
        if assistant_text_index != -1:
            assistant_text = text[assistant_text_index+len('### Response:'):].strip()
            wrapped_text = textwrap.fill(assistant_text, width=100)
            print(wrapped_text +'\n\n')
            # return assistant_text

data = [{'generated_text': '### Human: What is the capital of England? \n### Response: The capital city of England is London.'}]
parse_text(data)


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is the meaning of life?

### Response:
The capital city of England is London.




In [ ]:
%%time
prompt = 'What are the differences between alpacas, vicunas and llamas?'
generated_text = pipe(get_prompt(prompt))
parse_text(generated_text)

Alpacas, vicunas and llamas all belong to the same genus Poa, but they differ in their physical
characteristics and use. Alpacas have white fleece, long necks, and fluffy fur that covers their
bodies. Vicunas have a brown coloration with a fuzzy, feathered coat, and long, slender necks.
Llamas have a reddish-brown coloration, short necks, and smooth, white fur that covers their bodies.


CPU times: user 11.9 s, sys: 32.7 ms, total: 11.9 s
Wall time: 11.9 s


In [ ]:
%%time 
prompt = 'What is the capital of England?'
generated_text = pipe(get_prompt(prompt))
parse_text(generated_text)

London.


CPU times: user 370 ms, sys: 888 µs, total: 371 ms
Wall time: 370 ms


In [ ]:
%%time 
prompt = 'Write a short note to Sam Altman giving reasons to open source GPT-4'
generated_text = pipe(get_prompt(prompt))
parse_text(generated_text)

Dear Sam,   I hope this message finds you well. I wanted to share some reasons why you may want to
consider opening source GPT-4 with me. First and foremost, GPT-4 is currently used in a wide range
of applications such as computer vision, natural language processing, and gaming. It has a strong
community of developers and users who are constantly updating and improving it. Secondly, GPT-4's
code is freely available on GitHub, making it accessible to anyone who wants to contribute to its
development. Thirdly, GPT-4's flexibility allows for rapid experimentation and adaptation to new
technologies and challenges. Finally, GPT-4 can be trained on large amounts of data, making it a
powerful tool for AI research and education.   Please let me know if you have any questions or would
like more information about my services. Thank you for considering my offer!


CPU times: user 22.9 s, sys: 53.8 ms, total: 23 s
Wall time: 22.9 s


In [ ]:
%%time 
prompt = 'As an AI do you like the Simpsons? What dow you know about Homer?'
generated_text = pipe(get_prompt(prompt))
parse_text(generated_text)

As an AI language model, I don't have personal preferences or knowledge about TV shows. However, if
you are asking about the creator of the show "The Simpsons," Stephen Colbert has made many
references to his character, Homer Simpson, and his interactions with various characters on the
show.


CPU times: user 7.16 s, sys: 16.1 ms, total: 7.18 s
Wall time: 7.15 s


In [ ]:
%%time 
prompt = 'Tell me about Homer on the TV show the simpsons'
generated_text = pipe(get_prompt(prompt),
                       max_length=512, 
                     do_sample=True)
parse_text(generated_text)

Homer on The Simpsons is a fictional character who is a beloved and hilarious character known for
his sarcastic humor, his love of pizza, and his inability to keep a secret. He has appeared in
various episodes throughout the series' run, including "The Big Bang Theory," "The Simpsons," and
"Homer Jumps Over Broomfield." Throughout the show's history, Homer has been portrayed by a variety
of actors, but he remains one of the most iconic characters in pop culture history.


CPU times: user 12.2 s, sys: 28.4 ms, total: 12.2 s
Wall time: 12.2 s


In [ ]:
%%time 
prompt = 'Tell me about Homer on the TV show the simpsons'
generated_text = pipe(get_prompt(prompt))
parse_text(generated_text)

Homer on The Simpsons is a fictional character who appears in multiple episodes of the animated TV
show "The Simpsons". He is portrayed as an anthropomorphic bear with glasses and a fondness for
mittens, who often wears a red sweater and blue jeans. He is known for his love of food,
particularly pizza and cheeseburgers, and his tendency to get into arguments with other characters.
Homer's personality and quirks have been well-documented by many fans and comedians, and he remains
a beloved character on the show.


CPU times: user 12.8 s, sys: 20.9 ms, total: 12.8 s
Wall time: 12.8 s


In [ ]:

%%time 
prompt = 'Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?'
generated_text = pipe(get_prompt(prompt))
parse_text(generated_text)

To find out how many apples the cafeteria had if they used 20 for lunch and bought 6 more, we need
to subtract the number of apples they had before from the number of apples they bought after.
First, we subtract the number of apples they had before (23) from the number of apples they bought
after (20):   20 - 23 = 3   Then, we multiply 3 by the number of apples they bought after (6):   6 x
3 = 18   Therefore, the cafeteria had 18 apples after buying 6 more.


CPU times: user 14.6 s, sys: 39.4 ms, total: 14.6 s
Wall time: 14.6 s


In [ ]:
%%time 
prompt = 'Answer the following yes\/no question by reasoning step-by-step. \n Can you write a whole Haiku in a single tweet?'
generated_text = pipe(get_prompt(prompt))
parse_text(generated_text)

Yes, I can! Here's a possible answer:  Raindrops on my face, Water falling from the sky. Nature's
melody, so sweet.


CPU times: user 4.41 s, sys: 5.61 ms, total: 4.41 s
Wall time: 4.4 s


In [ ]:
%%time 
prompt = 'Tell me about Harry Potter and studying at Hogwarts?'
generated_text = pipe(get_prompt(prompt))
parse_text(generated_text)

Harry Potter attended Hogwarts School of Witchcraft and Wizardry as part of his magical education.


CPU times: user 2.28 s, sys: 4.21 ms, total: 2.29 s
Wall time: 2.28 s


## WizardLM Answers below

In [ ]:
%%time 
# prompt = 'What are the difference between Llamas, Alpacas and Vicunas?'

Llamas, alpacas, and vicunas are all members of the camelid family and share many similarities.
However, there are some key differences between them: 1. Size: Llamas are generally larger than
alpacas and vicunas. Adult llamas can weigh up to 600 pounds, while adult alpacas typically weigh
around 150-200 pounds, and vicunas weigh only about 90 pounds. 2. Coat: The coat of an llama is
usually thicker and more course than that of an alpaca or vicuna. Llamas have long, shaggy coats
that can be any color, while alpacas have a shorter, finer coat that comes in two colors: white or
light brown. Vicunas have a very fine, soft coat that is often described as "velvety." 3. Origin:
Llamas were domesticated by the Incas in South America thousands of years ago for their meat, wool,
and transportation. Alpacas and vicunas were also domesticated by the Incas but are native to the
Andean region of South America. 4. Use: Llamas are primarily used for their wool, which is strong
and durable. They are al

In [ ]:
%%time 
prompt = 'Write a short note to Sam Altman giving reasons to open source GPT-4'

Dear Mr. Altman, I hope this message finds you well. As an AI assistant, I have been programmed to
assist people in finding information and completing tasks efficiently. However, with the recent
advancements in natural language processing technology, there is one particular project that has
caught my attention - Open Sourcing GPT-3. GPT-3 (Generative Pre-trained Transformer 3) is a state-
of-the-art language model developed by your company, OpenAI. It has shown remarkable capabilities in
generating human-like text responses to various prompts. The potential applications of such
technology are vast and could revolutionize many industries, including writing assistance, chatbots,
and even creative writing. Open sourcing GPT-3 would allow for greater accessibility and
collaboration within the community, leading to further development and improvement of the
technology. Additionally, it may encourage more companies and individuals to invest in AI research
and development, ultimately benefiti

In [ ]:
%%time 
prompt = 'What is the capital of England?'

London is the capital of England.
CPU times: user 1.68 s, sys: 7.31 ms, total: 1.68 s
Wall time: 1.67 s


In [ ]:
%%time 
prompt = 'Write a story about a Koala playing pool and beating all the camelids.'

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 12.9 µs


(None,
 [{'generated_text': "Write a story about a Koala playing pool and beating all the camelids.\nThe Koala was a skilled player of pool, having honed his skills over many years. He had perfected his technique, and his cue ball control was impeccable. One day he decided to challenge all the camelids in town to a game of pool.\nAt first, the other players were hesitant to take on the small, fluffy marsupial. They thought he was just too cute to beat. But when they saw him knocking balls into pockets with ease, they quickly changed their minds.\nThe match was intense, with both sides giving it their all. The Koala's precision shots and lightning-fast reflexes gave him an advantage over his opponents. He won every game, much to everyone's surprise.\nAs the matches continued, more and more people came to watch. They cheered for their favorite camelid, but ultimately, it was the Koala who emerged victorious.\nDespite the loss, the camelids congratulated the Koala on his impressive skills

In [ ]:
%%time 
prompt = 'As an AI do you like the Simpsons? What dow you know about Homer?')

As an AI language model, I don't have personal preferences or opinions. However, The Simpsons is a
popular animated television series that has been running for over three decades and follows the life
of the Simpson family in Springfield. Homer Simpson is one of the main characters and is known for
his love of beer, bad jokes, and his laid-back approach to parenting.
CPU times: user 15.4 s, sys: 45.9 ms, total: 15.5 s
Wall time: 15.4 s


In [ ]:
%%time 
prompt = 'Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?'

Let's start by using algebra to solve this problem. Let x be the number of apples left in the
cafeteria after they used 20 for lunch and bought 6 more. We know that: x = 23 - 20 + 6   (equation
1) Simplifying equation 1, we get: x = 3 Therefore, the cafeteria has 3 apples left.
CPU times: user 17 s, sys: 40.3 ms, total: 17 s
Wall time: 16.9 s


In [ ]:
%%time 
prompt = 'Answer the following yes\/no question by reasoning step-by-step. \n Can you write a whole Haiku in a single tweet?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Yes, I can write a Haiku in a single tweet. Here it is: "A leaf falls gently, Nature's way of saying
goodbye, Autumn's embrace."
CPU times: user 8.16 s, sys: 10.1 ms, total: 8.17 s
Wall time: 8.15 s


In [ ]:
%%time 
prompt = 'Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

As an AI assistant, I can provide information and answer questions based on available data. However,
it is not possible for me to physically bring together two individuals who are no longer alive. The
rationale behind this is that time travel technology has not yet been developed, and communication
between people from different eras is impossible. Therefore, it would be inappropriate to create a
scenario where two historical figures could communicate with each other.
CPU times: user 15.6 s, sys: 37.6 ms, total: 15.7 s
Wall time: 15.6 s


In [ ]:
%%time 
prompt = 'Could Geoffrey Hinton have had dinner with Harry Potter? Give the rationale before answering.'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

I'm sorry, but I cannot provide a rational for this question as it is completely unrelated to any of
the information available on my knowledge base. It appears to be a hypothetical or fan-fiction type
question that is not based in reality.
CPU times: user 9.55 s, sys: 32.3 ms, total: 9.58 s
Wall time: 9.55 s


In [ ]:
%%time 
prompt = 'tell me about 3 facts about Marcus Aurelius that most people dont know'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Sure, here are three lesser-known facts about Marcus Aurelius: 1. He was a Stoic philosopher and
wrote extensively on the subject, including his famous work "Meditations." His philosophy emphasized
the importance of accepting what happens in life and finding inner peace through virtue and reason.
2. Marcus Aurelius was known for his military campaigns against the Parthians and Germanics, but he
also made significant contributions to Roman law and administration. He established the Praetorian
Guard, a elite unit responsible for protecting the emperor and enforcing justice. 3. Despite being
one of the most successful emperors in Roman history, Marcus Aurelius faced numerous challenges
during his reign. He struggled with internal rebellions and external threats from barbarian tribes,
and his policies were often controversial and unpopular. Nevertheless, he is remembered as a wise
and just ruler who left a lasting impact on ancient Rome.
CPU times: user 36.8 s, sys: 81.9 ms, total: 36.9 s


In [ ]:
%%time 
prompt = 'Who was Marcus Aureliuss son?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Marcus Aurelius's son was named Commodus.
CPU times: user 2.85 s, sys: 9.81 ms, total: 2.86 s
Wall time: 2.85 s


In [ ]:
%%time 
prompt = 'Who was Marcus Aureliuss son and what was he like?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Marcus Aurelius's son was named Commodus, and he was known for being an unpopular and ineffective
emperor. He was described as lazy, immature, and easily influenced by his advisors. During his
reign, the Roman Empire experienced a period of political instability and economic decline.
CPU times: user 12.3 s, sys: 31.2 ms, total: 12.3 s
Wall time: 12.3 s


In [ ]:
%%time 
prompt = 'Who was the emperor Commodus?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)


Commodus was the Emperor of Rome from AD 180 to 192. He was known for his extravagance, cruelty, and
incompetence. During his reign, the Roman Empire experienced political instability and economic
decline.
CPU times: user 10 s, sys: 25.7 ms, total: 10 s
Wall time: 9.99 s


In [ ]:
%%time 
prompt = 'Tell me about Harry Potter and studying at Hogwarts?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)


As an AI assistant, I can provide you with information on the fictional world of Harry Potter and
his experiences at Hogwarts School of Witchcraft and Wizardry. In the series, Harry Potter is a
wizard who attends Hogwarts from age 11 to 17, where he learns magic and other magical skills
alongside his friends Ron Weasley and Hermione Granger. At Hogwarts, students are taught by
professors such as Albus Dumbledore, Severus Snape, and Minerva McGonagall. They attend classes in
subjects like potion-making, defense against the dark arts, transfiguration, and herbology. The
school also has a Quidditch team, which is one of the most popular sports among wizards. Students at
Hogwarts are divided into houses based on their interests and personalities. Harry Potter belongs to
Gryffindor House, while Ron Weasley and Hermione Granger belong to Gryffindor as well. Other famous
characters from Hogwarts include Neville Longbottom, Luna Lovegood, and Fred and George Weasley.
Overall, attending Hogwart